# Team 2
# Assignment 3 - Part II

## Modules

In [2]:
!pip install graphviz

  Using cached graphviz-0.16-py2.py3-none-any.whl (19 kB)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp
from scipy import stats
import networkx as nx
import string
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import graphviz

sns.set()

## Load Data

In [5]:
# Demand (required bikes) of each Hub
dfDemands = pd.read_csv("demand_partII.csv", index_col=0)
dfDemands.reset_index(inplace=True)
dfDemands.head()

,Node,Demand
0,1,-14
1,2,-7
2,3,8
3,4,5
4,5,9


In [6]:
# Distances
dfDistances = pd.read_excel("distances_partII_new.xlsx", index_col=0)
dfDistances.reset_index(inplace=True)
dfDistances.head()

,i,j,Distance
0,1,1,0.000
1,2,1,3.223
2,3,1,2.743
3,4,1,2.206
4,5,1,0.310


In [7]:
distances_matrix = pd.read_csv("distances_partII.csv", header = None)

## Sets

In [8]:
# 12 Hubs
hubs = list(dfDemands["Node"])
numhubs = len(hubs)

timesteps = list(range(1,numhubs+1))
numtimesteps = len(timesteps)


# Manual Network Graph

In [17]:
test = pd.read_csv("distances_partII.csv")
test.head()

,1,2,3,4,5,6,7,8,9,10,...,32,33,34,35,36,37,38,39,40,41
0,0.000,3.22300,2.743,2.20600,0.310,1.964000,1.745640,2.01384,3.32200,1.62851,...,3.179000,0.468001,2.497000,6.87326,3.10616,2.95782,2.60770,6.98298,3.646870,6.56154
1,3.223,0.00000,5.966,1.01701,3.533,5.187000,1.633650,3.89271,2.50202,2.09880,...,0.044011,3.691000,5.720000,7.22446,5.63624,2.46950,2.17391,6.96960,0.822271,8.57780
2,2.743,5.96600,0.000,4.94900,2.433,0.779001,4.442790,3.31551,5.61113,4.18722,...,5.922000,2.275000,0.246008,7.71044,2.68509,5.23330,4.99450,8.08353,6.357220,5.78697
3,2.206,1.01701,4.949,0.00000,2.516,4.170000,0.736384,3.06563,2.34780,1.26643,...,0.973005,2.674000,4.703000,6.95370,4.74678,2.16350,1.76873,6.80866,1.553160,7.85336
4,0.310,3.53300,2.433,2.51600,0.000,1.654000,2.044350,2.02122,3.54859,1.89198,...,3.489000,0.158003,2.187000,6.91862,2.93572,3.17819,2.84544,7.06270,3.950920,6.42018


In [18]:
#CREATES NETWORK GRAPH BASED ON RELATIVE DISTANCE
dt = [('len', float)]
A = (test.to_numpy()*2).view(dt)

plt.figure(figsize=(20, 10))
G = nx.from_numpy_matrix(A)
# G = nx.relabel_nodes(G, dict(zip(range(len(G.nodes())),string.ascii_uppercase)))    

G = nx.drawing.nx_agraph.to_agraph(G)

G.node_attr.update(color="none", style="filled")
G.edge_attr.update(color="none", width="0.00001", bbox = dict(alpha = 0))

G.draw('output.png', format='png', prog='neato')

<Figure size 1440x720 with 0 Axes>

In [8]:
#Manually Select Clusters based on network graph above
tour1_hubs = [0,4,32,13,5,29,33,2,35,16,14]
tour1_hubs = [x+1 for x in tour1_hubs]
print(tour1_hubs)

tour2_hubs = [0,12,10,40,21,19,26,34,22,24,7,20]
tour2_hubs = [x+1 for x in tour2_hubs]
print(tour2_hubs)

tour3_hubs = [0,38,23,11,25,8,36,37,9,6,28]
tour3_hubs = [x+1 for x in tour3_hubs]
print(tour3_hubs)

tour4_hubs = [0,30,3,18,15,31,1,27,17,39]
tour4_hubs = [x+1 for x in tour4_hubs]
print(tour4_hubs)


[1, 5, 33, 14, 6, 30, 34, 3, 36, 17, 15]
[1, 13, 11, 41, 22, 20, 27, 35, 23, 25, 8, 21]
[1, 39, 24, 12, 26, 9, 37, 38, 10, 7, 29]
[1, 31, 4, 19, 16, 32, 2, 28, 18, 40]


In [9]:
print(dfDemands[dfDemands['Node'].isin(tour1_hubs)]['Demand'].sum())
print(dfDemands[dfDemands['Node'].isin(tour2_hubs)]['Demand'].sum())
print(dfDemands[dfDemands['Node'].isin(tour3_hubs)]['Demand'].sum())
print(dfDemands[dfDemands['Node'].isin(tour4_hubs)]['Demand'].sum())

-11
-6
-9
-14


In [10]:
check = tour1_hubs + tour2_hubs + tour3_hubs + tour4_hubs
check.sort()

In [11]:
timesteps_tour1 = list(range(1,len(tour1_hubs)+1))
numtimesteps_tour1 = len(timesteps_tour1)
print(timesteps_tour1)

timesteps_tour2 = list(range(1,len(tour2_hubs)+1))
numtimesteps_tour2 = len(timesteps_tour2)
print(timesteps_tour2)

timesteps_tour3 = list(range(1,len(tour3_hubs)+1))
numtimesteps_tour3 = len(timesteps_tour3)
print(timesteps_tour3)

timesteps_tour4 = list(range(1,len(tour4_hubs)+1))
numtimesteps_tour4 = len(timesteps_tour4)
print(timesteps_tour4)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [12]:
tour_list = []
tour_list.append(tour1_hubs)
tour_list.append(tour2_hubs)
tour_list.append(tour3_hubs)
tour_list.append(tour4_hubs)
print(tour_list)
print('\n')

timesteps_list = []
timesteps_list.append(timesteps_tour1)
timesteps_list.append(timesteps_tour2)
timesteps_list.append(timesteps_tour3)
timesteps_list.append(timesteps_tour4)
print(timesteps_list)

[[1, 5, 33, 14, 6, 30, 34, 3, 36, 17, 15], [1, 13, 11, 41, 22, 20, 27, 35, 23, 25, 8, 21], [1, 39, 24, 12, 26, 9, 37, 38, 10, 7, 29], [1, 31, 4, 19, 16, 32, 2, 28, 18, 40]]


[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]


In [13]:
for tour in range(4):

    # Parameters: Hub i to j distance (dictionary)
    hubsdistance = {}
    for i in tour_list[tour]:
        for j in tour_list[tour]:
            hubsdistance[(i,j)] = float(dfDistances[(dfDistances["i"] == i)&(dfDistances["j"] == j)]["Distance"])
    
    # Parameters: Demand in hub j 
    requiredbike = {}
    for j in tour_list[tour]:
        requiredbike[j] = float(dfDemands[(dfDemands["Node"]==j)]["Demand"])
    
    # Variables: if truck transferred form i to j in time t
    xvar = pulp.LpVariable.dict("x", (hubs, hubs, timesteps), cat=pulp.LpBinary)
    
    # Variables: number of bikes in the truck at time t
    cvar = pulp.LpVariable.dict("c", (timesteps), lowBound=0.0,upBound=30.0, cat=pulp.LpInteger)
    
    # Initialize model and objective sense
    Model = pulp.LpProblem(name="Model", sense=pulp.LpMinimize)
    
    # Objective: Total distance
    obj = pulp.lpSum([hubsdistance[(i,j)]*xvar[i,j,t] for i in tour_list[tour] for j in tour_list[tour] for t in timesteps_list[tour]])
    
    # Add objective function to the model
    Model += obj
    
    # Contraint: Every hub just visited once
    for j in hubs:
        Model += pulp.lpSum( [xvar[(i,j,t)] for t in timesteps_list[tour] for i in tour_list[tour]] ) == 1
    
    # Constraint: Conversation of flow
    for j in tour_list[tour]:
        for t in timesteps_list[tour]:
             for t in timesteps_list[tour][:len(timesteps_list[tour])-1]:
                Model += pulp.lpSum( [xvar[(i,j,t)] for i in tour_list[tour]] ) == pulp.lpSum( [xvar[(j,k,t+1)] for k in tour_list[tour]] )
    
    # Constraint: Truck Starts and ends at the Central Depot

    # At t=1, the only outflow is from Central Depot
    Model += pulp.lpSum( [xvar[(1,j,1)] for j in tour_list[tour]] ) == 1 
   # for i in tour_list[tour][1:]:
    #    Model += pulp.lpSum( [xvar[(i,j,1)] for j in tour_list[tour]] ) == 0

    # At t= final, the only inflow is to Central Depot
    Model += pulp.lpSum( [xvar[(i,1,len(timesteps_list[tour]))] for i in tour_list[tour]] ) == 1 
  #  for j in tour_list[tour][1:]:
   #     Model += pulp.lpSum( [xvar[(i,j,len(timesteps_list[tour]))] for i in tour_list[tour] ]) == 0
    
    # Constraint: Capacity of Truck is maximum 30 bikes

    # At beginning of day, truck carries 0 bikes
    Model += cvar[1] == 0

    # Capacity of truck changes based on bike demand previous period
    for t in timesteps_list[tour][:len(timesteps_list[tour])-1]:
        Model += cvar[t+1] == cvar[t] + pulp.lpSum([requiredbike[j]*xvar[(i,j,t)] for i in tour_list[tour] for j in tour_list[tour]])
    
    
    # Write LP to file (optional, but often good to inspect model and find errors)
    Model.writeLP("Model.lp")

    # Solve model
    Model.solve()
    print("Status:", pulp.LpStatus[Model.status])
    
    # Total cost
    totalDistance = pulp.value(Model.objective)
    print("Total distance: " + str(totalDistance))
    
    for t in timesteps_list[tour]:
        for i in tour_list[tour]:
            for j in tour_list[tour]:
                if xvar[(i,j,t)].varValue == 1.0:
                    print("Timestep " + str(t) +": Hub " + str(i) +  " to Hub " + str(j) + 
                          " \n\t    truck capacity: " + str(cvar[t].varValue) +
                          " \n\t    distance travelled: " + str(hubsdistance[(i,j)]))  
    print('\n\n')

Status: Optimal
Total distance: 13.242276999999998
Timestep 1: Hub 1 to Hub 15 
	    truck capacity: 0.0 
	    distance travelled: 1.832
Timestep 2: Hub 15 to Hub 17 
	    truck capacity: 15.0 
	    distance travelled: 1.03
Timestep 3: Hub 17 to Hub 5 
	    truck capacity: 10.0 
	    distance travelled: 1.11201
Timestep 4: Hub 5 to Hub 14 
	    truck capacity: 19.0 
	    distance travelled: 0.680001
Timestep 5: Hub 14 to Hub 33 
	    truck capacity: 11.0 
	    distance travelled: 0.522
Timestep 6: Hub 33 to Hub 6 
	    truck capacity: 26.0 
	    distance travelled: 1.496
Timestep 7: Hub 6 to Hub 30 
	    truck capacity: 17.0 
	    distance travelled: 0.212002
Timestep 8: Hub 30 to Hub 34 
	    truck capacity: 10.0 
	    distance travelled: 0.321006
Timestep 9: Hub 34 to Hub 3 
	    truck capacity: 5.0 
	    distance travelled: 0.246008
Timestep 10: Hub 3 to Hub 36 
	    truck capacity: 13.0 
	    distance travelled: 2.68509
Timestep 11: Hub 36 to Hub 1 
	    truck capacity: 3.0 
	    d

# Clustering

In [14]:
#Create Dataframe of different routes based on clustering
#distance formula is in a matrix form allowing for clustering
X = distances_matrix.to_numpy()
#Create empty dataframe to append
final_tour_clusters=pd.DataFrame()
#AGGLOMERATIVE CLUSTERING
for clusters in range(2,11):
    #Use Agglomerative clustering algorithm where K is 2:10
    model_cluster = AgglomerativeClustering(n_clusters=clusters,affinity='precomputed', linkage ='average').fit(X)

    #Create temporary dataframe where cluster labels are used as columns
    tours_clusters = pd.DataFrame(model_cluster.labels_)

    #adds columns based on the number of clusters created
    for i in range(1,clusters+1):
            tours_clusters[i] = ""

    #Checks if the clustered routes include hubs or node 0-40
    for i in range(1,clusters+1):
        for j in range(41):
            # if hub is in the cluster than it equals one
            if tours_clusters.iloc[(j,0)] == i-1:
                tours_clusters.iloc[(j,i)] = 1
            else:
                tours_clusters.iloc[(j,i)] = 0
    
    #Transpose dataframe to make cluster labels the rows so each row is a different tour
    tours_clusters = tours_clusters.transpose()
    tours_clusters = tours_clusters.drop([0])
    #Add the temprary table to dataframe and redo process for another k value
    final_tour_clusters= final_tour_clusters.append(tours_clusters)
    

In [15]:
#KMEANS CLUSTERING
for clusters in range(2,11):
    #Use KMeans clustering algorithm where K is 2:10
    model_cluster = KMeans(n_clusters=clusters, random_state=101).fit(X)

    #Create temporary dataframe where cluster labels are used as columns
    tours_clusters = pd.DataFrame(model_cluster.labels_)

    #adds columns based on the number of clusters created
    for i in range(1,clusters+1):
            tours_clusters[i] = ""

    #Checks if the clustered routes include hubs or node 0-40
    for i in range(1,clusters+1):
        for j in range(41):
            # if hub is in the cluster than it equals one
            if tours_clusters.iloc[(j,0)] == i-1:
                tours_clusters.iloc[(j,i)] = 1
            else:
                tours_clusters.iloc[(j,i)] = 0
    
    #Transpose dataframe to make cluster labels the rows so each row is a different tour
    tours_clusters = tours_clusters.transpose()
    tours_clusters = tours_clusters.drop([0])
    #Add the temprary table to dataframe and redo process for another k value
    final_tour_clusters= final_tour_clusters.append(tours_clusters)

In [16]:
final_tour_clusters

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
1,0,1,0,1,0,0,1,1,1,1,...,1,0,0,0,1,1,1,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
3,1,0,1,0,1,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
7,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
8,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [16]:
# Make sure every route selected starts with 1 (Node 0 should equal 1)
final_tour_clusters.iloc[:,0] = 1

# Create counts column to count how many hubs are in a tour
final_tour_clusters['num_hubs_visited'] = final_tour_clusters.sum(axis=1)

# Reset index as they are currently cluster labels
final_tour_clusters = final_tour_clusters.reset_index(drop=True)

# compute net demand for each tour in full dataframe
for tour in range(0,final_tour_clusters.shape[0]):
    #create temporary demands data table
    demand_temp = dfDemands.copy()
    #for each column sum all the demand to find the net demand at the end
    for x in range(41): 
        if final_tour_clusters.iloc[tour,x] == 0:
            demand_temp = demand_temp.drop([x])   
    demand_temp = demand_temp.drop(columns = ['Node'])
    final_tour_clusters.loc[tour,'net_demand'] = demand_temp.sum().Demand

In [17]:
#Remove all tours where net demand is greather than 30 or less than 30
#This is because there is a 30 cpacity constraint and will not be feasible if above this number
final_tour_clusters = final_tour_clusters.drop(final_tour_clusters[(final_tour_clusters['net_demand']>30)].index) 
final_tour_clusters = final_tour_clusters.drop(final_tour_clusters[(final_tour_clusters['net_demand']<-30)].index) 
#Only allow routes that have more than two hubs included
final_tour_clusters=final_tour_clusters[final_tour_clusters["num_hubs_visited"]>=2]
final_tour_clusters=final_tour_clusters[final_tour_clusters["num_hubs_visited"]<=16]
#Reset index again
final_tour_clusters = final_tour_clusters.reset_index(drop=True)

In [18]:
final_tour_clusters

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,num_hubs_visited,net_demand
0,1,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,8.0,-11.0
1,1,0,0,0,0,0,0,1,1,0,...,0,0,1,1,1,0,0,0,13.0,8.0
2,1,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,8.0,-11.0
3,1,1,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,15.0,9.0
4,1,0,0,0,0,0,0,1,1,0,...,0,0,1,1,1,0,0,0,13.0,8.0
5,1,1,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,15.0,9.0
6,1,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,8.0,-11.0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,3.0,-27.0
8,1,1,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,15.0,9.0
9,1,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,8.0,-11.0


In [19]:
#Only select the hubs
final_tour_clusters=final_tour_clusters[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]]

In [20]:
#Repeat manual steps
#add the routes into an array to redo part 1
tour_list = []
timesteps_list = []
for i in range(0,len(final_tour_clusters)):
    tour_hubs=final_tour_clusters.loc[[i]].replace(0,np.nan).dropna(axis=1,how="all")
    tour_hubs=tour_hubs.columns.tolist()
    tour_hubs = [x+1 for x in tour_hubs]
    tour_list.append(tour_hubs)

    timesteps_tour1 = list(range(1,len(tour_hubs)+1))
    timesteps_list.append(timesteps_tour1)

In [21]:
print(timesteps_list)
print(" ")
print(tour_list)

[[1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 4], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3, 4], [1, 2, 3], [1, 2, 3, 4, 5], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3, 4], [1, 2], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 

In [22]:
#Repeat same steps as Manual section
for tour in range(0,len(tour_list)):
    
    # Parameters: Hub i to j distance (dictionary)
    hubsdistance = {}
    for i in tour_list[tour]:
        for j in tour_list[tour]:
            hubsdistance[(i,j)] = float(dfDistances[(dfDistances["i"] == i)&(dfDistances["j"] == j)]["Distance"])
    
    # Parameters: Demand in hub j 
    requiredbike = {}
    for j in tour_list[tour]:
        requiredbike[j] = float(dfDemands[(dfDemands["Node"]==j)]["Demand"])
    
    # Variables: if truck transferred form i to j in time t
    xvar = pulp.LpVariable.dict("x", (hubs, hubs, timesteps), cat=pulp.LpBinary)
    
    # Variables: number of bikes in the truck at time t
    cvar = pulp.LpVariable.dict("c", (timesteps), lowBound=0.0,upBound=30.0, cat=pulp.LpInteger)
    
    # Initialize model and objective sense
    Model = pulp.LpProblem(name="Model", sense=pulp.LpMinimize)
    
    # Objective: Total distance
    obj = pulp.lpSum([hubsdistance[(i,j)]*xvar[i,j,t] for i in tour_list[tour] for j in tour_list[tour] for t in timesteps_list[tour]])
    
    # Add objective function to the model
    Model += obj
    
    # Contraint: Every hub just visited once
    for j in hubs:
        Model += pulp.lpSum( [xvar[(i,j,t)] for t in timesteps_list[tour] for i in tour_list[tour]] ) == 1
    
    # Constraint: Conversation of flow
    for j in tour_list[tour]:
        for t in timesteps_list[tour]:
             for t in timesteps_list[tour][:len(timesteps_list[tour])-1]:
                Model += pulp.lpSum( [xvar[(i,j,t)] for i in tour_list[tour]] ) == pulp.lpSum( [xvar[(j,k,t+1)] for k in tour_list[tour]] )
    
    # Constraint: Truck Starts and ends at the Central Depot

    # At t=1, the only outflow is from Central Depot
    Model += pulp.lpSum( [xvar[(1,j,1)] for j in tour_list[tour]] ) == 1 
   # for i in tour_list[tour][1:]:
    #    Model += pulp.lpSum( [xvar[(i,j,1)] for j in tour_list[tour]] ) == 0

    # At t= final, the only inflow is to Central Depot
    Model += pulp.lpSum( [xvar[(i,1,len(timesteps_list[tour]))] for i in tour_list[tour]] ) == 1 
  #  for j in tour_list[tour][1:]:
   #     Model += pulp.lpSum( [xvar[(i,j,len(timesteps_list[tour]))] for i in tour_list[tour] ]) == 0
    
    # Constraint: Capacity of Truck is maximum 30 bikes

    # At beginning of day, truck carries 0 bikes
    Model += cvar[1] == 0

    # Capacity of truck changes based on bike demand previous period
    for t in timesteps_list[tour][:len(timesteps_list[tour])-1]:
        Model += cvar[t+1] == cvar[t] + pulp.lpSum([requiredbike[j]*xvar[(i,j,t)] for i in tour_list[tour] for j in tour_list[tour]])
    
    
    # Write LP to file (optional, but often good to inspect model and find errors)
    Model.writeLP("Model.lp")
    print("Tour: ",tour)
    # Solve model
    Model.solve()
    print("Status:", pulp.LpStatus[Model.status])
    
    # Total cost
    totalDistance = pulp.value(Model.objective)
    print("Total distance: " + str(totalDistance))
    
    for t in timesteps_list[tour]:
        for i in tour_list[tour]:
            for j in tour_list[tour]:
                if xvar[(i,j,t)].varValue == 1.0:
                    print("Timestep " + str(t) +": Hub " + str(i) +  " to Hub " + str(j) + 
                          " \n\t    truck capacity: " + str(cvar[t].varValue) +
                          " \n\t    distance travelled: " + str(hubsdistance[(i,j)]))  
    print('\n\n')
    
    #Add informatio on the optimal total distance of each route
    final_tour_clusters.loc[tour,'distance'] = totalDistance
    #Add information if the route is infeasible or not
    final_tour_clusters.loc[tour,'optimal'] = pulp.LpStatus[Model.status]

Tour:  0
Status: Optimal
Total distance: 5.486013
Timestep 1: Hub 1 to Hub 5 
	    truck capacity: 0.0 
	    distance travelled: 0.31
Timestep 2: Hub 5 to Hub 33 
	    truck capacity: 9.0 
	    distance travelled: 0.158003
Timestep 3: Hub 33 to Hub 14 
	    truck capacity: 24.0 
	    distance travelled: 0.522
Timestep 4: Hub 14 to Hub 34 
	    truck capacity: 16.0 
	    distance travelled: 1.507
Timestep 5: Hub 34 to Hub 3 
	    truck capacity: 11.0 
	    distance travelled: 0.246008
Timestep 6: Hub 3 to Hub 30 
	    truck capacity: 19.0 
	    distance travelled: 0.567
Timestep 7: Hub 30 to Hub 6 
	    truck capacity: 12.0 
	    distance travelled: 0.212002
Timestep 8: Hub 6 to Hub 1 
	    truck capacity: 3.0 
	    distance travelled: 1.964





/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Tour:  1
Status: Optimal
Total distance: 16.724421
Timestep 1: Hub 1 to Hub 8 
	    truck capacity: 0.0 
	    distance travelled: 2.01384
Timestep 2: Hub 8 to Hub 12 
	    truck capacity: 9.0 
	    distance travelled: 1.8244
Timestep 3: Hub 12 to Hub 23 
	    truck capacity: 15.0 
	    distance travelled: 0.843367
Timestep 4: Hub 23 to Hub 36 
	    truck capacity: 30.0 
	    distance travelled: 2.2435
Timestep 5: Hub 36 to Hub 11 
	    truck capacity: 20.0 
	    distance travelled: 1.54271
Timestep 6: Hub 11 to Hub 13 
	    truck capacity: 10.0 
	    distance travelled: 0.952064
Timestep 7: Hub 13 to Hub 25 
	    truck capacity: 0.0 
	    distance travelled: 1.29371
Timestep 8: Hub 25 to Hub 21 
	    truck capacity: 10.0 
	    distance travelled: 0.798164
Timestep 9: Hub 21 to Hub 26 
	    truck capacity: 20.0 
	    distance travelled: 0.598206
Timestep 10: Hub 26 to Hub 9 
	    truck capacity: 27.0 
	    distance travelled: 1.20965
Timestep 11: Hub 9 to Hub 37 
	    truck capacity: 17

Tour:  9
Status: Optimal
Total distance: 5.486013
Timestep 1: Hub 1 to Hub 5 
	    truck capacity: 0.0 
	    distance travelled: 0.31
Timestep 2: Hub 5 to Hub 33 
	    truck capacity: 9.0 
	    distance travelled: 0.158003
Timestep 3: Hub 33 to Hub 14 
	    truck capacity: 24.0 
	    distance travelled: 0.522
Timestep 4: Hub 14 to Hub 34 
	    truck capacity: 16.0 
	    distance travelled: 1.507
Timestep 5: Hub 34 to Hub 3 
	    truck capacity: 11.0 
	    distance travelled: 0.246008
Timestep 6: Hub 3 to Hub 30 
	    truck capacity: 19.0 
	    distance travelled: 0.567
Timestep 7: Hub 30 to Hub 6 
	    truck capacity: 12.0 
	    distance travelled: 0.212002
Timestep 8: Hub 6 to Hub 1 
	    truck capacity: 3.0 
	    distance travelled: 1.964



Tour:  10
Status: Infeasible
Total distance: 102.548776217714
Timestep 2: Hub 22 to Hub 41 
	    truck capacity: 0.0 
	    distance travelled: 1.53492



Tour:  11
Status: Optimal
Total distance: 5.486013
Timestep 1: Hub 1 to Hub 5 
	    truck ca

Tour:  25
Status: Infeasible
Total distance: 281.44471227445655
Timestep 2: Hub 27 to Hub 39 
	    truck capacity: 12.272727 
	    distance travelled: 0.953843
Timestep 3: Hub 20 to Hub 39 
	    truck capacity: 0.0 
	    distance travelled: 1.96129
Timestep 4: Hub 1 to Hub 20 
	    truck capacity: 292.09091 
	    distance travelled: 6.18171
Timestep 4: Hub 1 to Hub 35 
	    truck capacity: 292.09091 
	    distance travelled: 6.87326
Timestep 4: Hub 1 to Hub 39 
	    truck capacity: 292.09091 
	    distance travelled: 6.98298
Timestep 4: Hub 20 to Hub 39 
	    truck capacity: 292.09091 
	    distance travelled: 1.96129
Timestep 4: Hub 27 to Hub 35 
	    truck capacity: 292.09091 
	    distance travelled: 0.239201
Timestep 4: Hub 27 to Hub 39 
	    truck capacity: 292.09091 
	    distance travelled: 0.953843
Timestep 5: Hub 39 to Hub 20 
	    truck capacity: 0.0 
	    distance travelled: 1.96129



Tour:  26
Status: Infeasible
Total distance: 13.59989768
Timestep 1: Hub 1 to Hub 1 
	    

Tour:  38
Status: Infeasible
Total distance: 51.479015724555005
Timestep 2: Hub 3 to Hub 34 
	    truck capacity: 0.0 
	    distance travelled: 0.246008
Timestep 4: Hub 1 to Hub 3 
	    truck capacity: -93.0 
	    distance travelled: 2.743
Timestep 4: Hub 34 to Hub 3 
	    truck capacity: -93.0 
	    distance travelled: 0.246008
Timestep 5: Hub 3 to Hub 30 
	    truck capacity: 0.0 
	    distance travelled: 0.567
Timestep 5: Hub 34 to Hub 1 
	    truck capacity: 0.0 
	    distance travelled: 2.497
Timestep 5: Hub 34 to Hub 6 
	    truck capacity: 0.0 
	    distance travelled: 0.533001
Timestep 5: Hub 34 to Hub 30 
	    truck capacity: 0.0 
	    distance travelled: 0.321006



Tour:  39
Status: Optimal
Total distance: 14.641768000000003
Timestep 1: Hub 1 to Hub 8 
	    truck capacity: 0.0 
	    distance travelled: 2.01384
Timestep 2: Hub 8 to Hub 21 
	    truck capacity: 9.0 
	    distance travelled: 1.61819
Timestep 3: Hub 21 to Hub 12 
	    truck capacity: 19.0 
	    distance travelle

Tour:  42
Status: Infeasible
Total distance: 68.12237350000001
Timestep 2: Hub 23 to Hub 1 
	    truck capacity: 36.0 
	    distance travelled: 3.46512
Timestep 4: Hub 1 to Hub 25 
	    truck capacity: 0.0 
	    distance travelled: 3.56972
Timestep 5: Hub 1 to Hub 13 
	    truck capacity: 0.0 
	    distance travelled: 3.94223
Timestep 5: Hub 25 to Hub 11 
	    truck capacity: 0.0 
	    distance travelled: 2.24361
Timestep 5: Hub 25 to Hub 13 
	    truck capacity: 0.0 
	    distance travelled: 1.29371
Timestep 5: Hub 25 to Hub 36 
	    truck capacity: 0.0 
	    distance travelled: 2.28672
Timestep 6: Hub 11 to Hub 1 
	    truck capacity: 0.0 
	    distance travelled: 4.50821
Timestep 6: Hub 36 to Hub 1 
	    truck capacity: 0.0 
	    distance travelled: 3.10616



Tour:  43
Status: Infeasible
Total distance: 51.479015724555005
Timestep 2: Hub 3 to Hub 34 
	    truck capacity: 0.0 
	    distance travelled: 0.246008
Timestep 4: Hub 1 to Hub 3 
	    truck capacity: -93.0 
	    distance tra

Tour:  57
Status: Infeasible
Total distance: 67.469472
Timestep 1: Hub 1 to Hub 17 
	    truck capacity: 0.0 
	    distance travelled: 0.802016
Timestep 2: Hub 17 to Hub 17 
	    truck capacity: 55.0 
	    distance travelled: 0.0
Timestep 3: Hub 17 to Hub 1 
	    truck capacity: 0.0 
	    distance travelled: 0.802016



Tour:  58
Status: Infeasible
Total distance: 102.548776217714
Timestep 2: Hub 22 to Hub 41 
	    truck capacity: 0.0 
	    distance travelled: 1.53492



Tour:  59
Status: Infeasible
Total distance: 13.59989768
Timestep 1: Hub 1 to Hub 1 
	    truck capacity: 0.0 
	    distance travelled: 0.0
Timestep 3: Hub 1 to Hub 37 
	    truck capacity: 0.0 
	    distance travelled: 2.95782
Timestep 4: Hub 37 to Hub 1 
	    truck capacity: 0.0 
	    distance travelled: 2.95782





In [24]:
#Filter out infeasible routes
final_df=final_tour_clusters[final_tour_clusters["optimal"]!="Infeasible"]

In [25]:
#Reset Index
final_df=final_df.reset_index(drop=True)

In [26]:
final_df

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,distance,optimal
0,1,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,5.486013,Optimal
1,1,0,0,0,0,0,0,1,1,0,...,0,0,1,1,1,0,0,0,16.724421,Optimal
2,1,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,5.486013,Optimal
3,1,1,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,10.006788,Optimal
4,1,0,0,0,0,0,0,1,1,0,...,0,0,1,1,1,0,0,0,16.724421,Optimal
5,1,1,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,10.006788,Optimal
6,1,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,5.486013,Optimal
7,1,1,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,10.006797,Optimal
8,1,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,5.486013,Optimal
9,1,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,5.486013,Optimal


In [27]:
#Add the manual clusters we created from the network graph
for i in range(41):
    final_df.loc[27,i]=0
    final_df.loc[28,i]=0
    final_df.loc[29,i]=0
    final_df.loc[30,i]=0


for j in range(0, len(tour1_hubs)):
    for i in range(1,41):
        if (tour1_hubs[j]==i-1):
            final_df.loc[27,i-2]=1

for j in range(0, len(tour2_hubs)):
    for i in range(41):
        if (tour2_hubs[j]==i-1):
            final_df.loc[28,i-2]=1

for j in range(0, len(tour3_hubs)):
    for i in range(41):
        if (tour3_hubs[j]==i-1):
            final_df.loc[29,i-2]=1

for j in range(0, len(tour4_hubs)):
    for i in range(41):
        if (tour4_hubs[j]==i-1):
            final_df.loc[30,i-2]=1

#Manual fixes to the routes  
final_df.loc[27,"distance"]=13.242276999999998
final_df.loc[28,"distance"]=22.267284999999998
final_df.loc[29,"distance"]=17.251826000000005
final_df.loc[30,"distance"]=8.5515624
final_df.loc[28,40]=1
final_df.loc[30,39]=1

In [28]:
13.242276999999998+22.267284999999998+17.251826000000005+8.5515624

61.3129504

In [29]:
final_df

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,distance,optimal
0,1.0,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,5.486013,Optimal
1,1.0,0,0,0,0,0,0,1,1,0,...,0,0,1,1,1,0,0,0,16.724421,Optimal
2,1.0,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,5.486013,Optimal
3,1.0,1,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,10.006788,Optimal
4,1.0,0,0,0,0,0,0,1,1,0,...,0,0,1,1,1,0,0,0,16.724421,Optimal
5,1.0,1,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,10.006788,Optimal
6,1.0,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,5.486013,Optimal
7,1.0,1,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,10.006797,Optimal
8,1.0,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,5.486013,Optimal
9,1.0,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,5.486013,Optimal


In [30]:
tour4_hubs

[1, 31, 4, 19, 16, 32, 2, 28, 18, 40]

In [31]:
# Problem formulation

tours = list(range(0,len(final_df)))

# Variables: Tour Selected or Not
xvar = pulp.LpVariable.dict("x", tours, lowBound=0.0, cat=pulp.LpBinary)

# Initialize model and objective sense
Model2 = pulp.LpProblem(name="Model2", sense=pulp.LpMinimize)

# Constraint: Only 4 tours selected as there are 4 trucks
Model2 += pulp.lpSum([xvar[(i)] for i in tours]) <= 4

# Constraint: All hubs must be visited based on the selected tours
for x in range(41):
    Model2 += pulp.lpSum([final_df.loc[i,x] * xvar[(i)] for i in tours]) >= 1
    
# Objective function
# minimize total distance of all tours together
obj = pulp.lpSum([ xvar[(i)] * final_df.loc[i,'distance'] for i in tours])

# add objective to model
Model2 +=obj

# Write LP to file (optional, but often good to inspect model and find errors)
Model2.writeLP("Model2.lp")

# Solve model
Model2.solve()

# Total cost
totalDistance = pulp.value(Model2.objective)
#print("Total Distance: " + str(totalCost))
   
print("Status:", pulp.LpStatus[Model2.status])
print("Total Distance:", str(totalDistance))

Status: Optimal
Total Distance: 61.3129504


In [32]:
#Print Tours Selected Distance
for i in tours:
    if xvar[(i)].varValue == 1.0:
        print("Tour",i,"Distance: ",final_df.loc[i,'distance'])

Tour 27 Distance:  13.242276999999998
Tour 28 Distance:  22.267284999999998
Tour 29 Distance:  17.251826000000005
Tour 30 Distance:  8.5515624


In [33]:
# Check All Hubs Are Covered
for x in range(41):
    trucks_no=0
    for i in tours:
        trucks_no =  trucks_no + final_df.loc[i,x] * xvar[(i)].varValue
    print("Hub",x+1,"had",trucks_no,'trucks')

Hub 1 had 4.0 trucks
Hub 2 had 1.0 trucks
Hub 3 had 1.0 trucks
Hub 4 had 1.0 trucks
Hub 5 had 1.0 trucks
Hub 6 had 1.0 trucks
Hub 7 had 1.0 trucks
Hub 8 had 1.0 trucks
Hub 9 had 1.0 trucks
Hub 10 had 1.0 trucks
Hub 11 had 1.0 trucks
Hub 12 had 1.0 trucks
Hub 13 had 1.0 trucks
Hub 14 had 1.0 trucks
Hub 15 had 1.0 trucks
Hub 16 had 1.0 trucks
Hub 17 had 1.0 trucks
Hub 18 had 1.0 trucks
Hub 19 had 1.0 trucks
Hub 20 had 1.0 trucks
Hub 21 had 1.0 trucks
Hub 22 had 1.0 trucks
Hub 23 had 1.0 trucks
Hub 24 had 1.0 trucks
Hub 25 had 1.0 trucks
Hub 26 had 1.0 trucks
Hub 27 had 1.0 trucks
Hub 28 had 1.0 trucks
Hub 29 had 1.0 trucks
Hub 30 had 1.0 trucks
Hub 31 had 1.0 trucks
Hub 32 had 1.0 trucks
Hub 33 had 1.0 trucks
Hub 34 had 1.0 trucks
Hub 35 had 1.0 trucks
Hub 36 had 1.0 trucks
Hub 37 had 1.0 trucks
Hub 38 had 1.0 trucks
Hub 39 had 1.0 trucks
Hub 40 had 1.0 trucks
Hub 41 had 1.0 trucks
